In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import savefig
import cv2
import glob as gl

In [ ]:
count = 1

path_hd = 'dataset/train/train_hd/*.jpg'
path_messy = 'dataset/train/train_messy/*.jpg'

paths_list_messy = [path_messy]
paths_list_hd = [path_hd]

for path in paths_list_messy:
    filenames = gl.glob(path)
    for filename in filenames:
       
        messy_img = cv2.imread(filename)
        messy_img = cv2.resize(messy_img, (128, 128))
    
        # mkdir if folder not exist
        cv2.imwrite("messy_images/messy_" +str(count) +".jpg", messy_img)
        count += 1

count = 1
for path in paths_list_hd:
    filenames = gl.glob(path)
    for filename in filenames:
       
        image = cv2.imread(filename)
        image = cv2.resize(image, (128, 128))
    
        cv2.imwrite("hd_images/hd_" +str(count) +".jpg", image)
        count += 1

In [ ]:
filenames = gl.glob("dataset/test/test_hd/*.jpg")

count = 1
for file in filenames:
    img = cv2.imread(file)
    img = cv2.resize(img, (128, 128))
    cv2.imwrite('actual_hd_test/hd_' + str(count) + '.jpg', img)
    count += 1

In [ ]:
num_images = 426

dataset = []

for i in range(1, num_images + 1):
    #print(i)
    img = cv2.imread("hd_images/hd_" +str(i) + ".jpg")
    #print(img)
    dataset.append(np.array(img))

dataset_source = np.asarray(dataset)
print(dataset_source.shape)

dataset_tar = []

for i in range(1, num_images + 1):
    img = cv2.imread("messy_images/messy_" +str(i) + ".jpg")
    #print(img)
    dataset_tar.append(np.array(img))

dataset_target = np.asarray(dataset_tar)
print(dataset_target.shape)

In [ ]:
def autoencoder(inputs):  # Undercomplete Autoencoder

    # Encoder
    net = tf.layers.conv2d(inputs, 128, 2, activation=tf.nn.relu)
    net = tf.layers.max_pooling2d(net, 2, 2, padding='same')
    print(net.shape)

    # Decoder
    net = tf.image.resize_nearest_neighbor(net, tf.constant([129, 129]))
    net = tf.layers.conv2d(net, 1, 2, activation=None, name='outputOfAuto')

    print(net.shape)

    return net

In [ ]:
#The “None” represents that the batch size would be determined at the runtime.
ae_inputs = tf.placeholder(tf.float32, (None, 128, 128, 3), name='inputToAuto')
ae_target = tf.placeholder(tf.float32, (None, 128, 128, 3))

ae_outputs = autoencoder(ae_inputs)
lr = 0.001  # learning rate 

loss = tf.reduce_mean(tf.square(ae_outputs - ae_target))
train_op = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)
# Intialize the network 
init = tf.global_variables_initializer()

In [ ]:
batch_size = 8
epoch_num = 1

saving_path = 'autoencoder_messy_to_hd/SavedModel/AutoencoderMessyToHd.ckpt'

saver_ = tf.train.Saver(max_to_keep=3)

batch_img = dataset_source[0:batch_size]
batch_out = dataset_target[0:batch_size]

num_batches = num_images // batch_size

sess = tf.Session()
sess.run(init)

for ep in range(epoch_num):
    batch_size = 0
    for batch_n in range(num_batches):  # batches loop

        _, c = sess.run([train_op, loss], feed_dict={ae_inputs: batch_img, ae_target: batch_out})
        print("Epoch: {} - cost = {:.5f}".format((ep + 1), c))

        batch_img = dataset_source[batch_size: batch_size + 32]
        batch_out = dataset_target[batch_size: batch_size + 32]

        batch_size += 32

    saver_.save(sess, saving_path, global_step=ep)

sess.close()

In [ ]:
saver = tf.train.Saver()

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

saver.restore(sess, 'autoencoder_messy_to_hd/SavedModel/AutoencoderMessyToHd.ckpt-49')

filenames = gl.glob('actual_hd_test/*.jpg')

test_data = []
for file in filenames[0:100]:
    test_data.append(np.array(cv2.imread(file)))

test_dataset = np.asarray(test_data)
print(test_dataset.shape)

# Running the test data on the autoencoder
batch_imgs = test_dataset
hd_imgs = sess.run(ae_outputs, feed_dict = {ae_inputs: batch_imgs})

for i in range(hd_imgs.shape[0]):
    cv2.imwrite('gen_hd_images/gen_hd_' +str(i) +'.jpg', hd_imgs[i])